# Introduction
This notebook is part of a machine learning for healthcare exercise, focusing on using the Responsible AI (RAI) package to enhance clinical decision-making. The toolkit will be used to analyze diabetes progression data, with two key objectives:

1. Plan real-world action through counterfactual and causal analysis: By leveraging counterfactual example analysis and causal inferencing, we will explore decision-making strategies based on diabetes progression data to understand potential treatment paths and their impacts.

2. Assess disease progression predictions: A regression model trained on diabetes progression data will be assessed, examining its performance in predicting disease progression to inform clinical decisions.

**The goal is for you to provide non-trivial insights for clinical decision making, leveraging machine learning, paired with responsible AI tools, to improve patients outcomes in the healthcare contexts.**

Based on notebooks from the [Responsible AI toolkit](https://github.com/microsoft/responsible-ai-toolbox/tree/main/notebooks)

In [29]:
import shap
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_breast_cancer
from interpret.ext.blackbox import TabularExplainer

Load the [breast cancer diagnosis data](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset) and specify the different types of features. Then, clean it and put it into a DataFrame with named columns.

Data Set Characteristics:

Number of Instances: 569

Number of Attributes: 30 numeric, predictive attributes and the class

Attribute Information:
- radius (mean of distances from center to points on the perimeter)

- texture (standard deviation of gray-scale values)

- perimeter

- area

- smoothness (local variation in radius lengths)

- compactness (perimeter^2 / area - 1.0)

- concavity (severity of concave portions of the contour)

- concave points (number of concave portions of the contour)

- symmetry

- fractal dimension (“coastline approximation” - 1)

The mean, standard error, and “worst” or largest (mean of the three worst/largest values) of these features were computed for each image, resulting in 30 features. For instance, field 0 is Mean Radius, field 10 is Radius SE, field 20 is Worst Radius.

class:
- WDBC-Malignant

- WDBC-Benign

In [30]:
breast_cancer_data = load_breast_cancer()
classes = breast_cancer_data.target_names.tolist()

In [41]:
type(breast_cancer_data)

sklearn.utils._bunch.Bunch

In [37]:
data = sklearn.datasets.load_breast_cancer()
target_feature = 'y'
continuous_features = data.feature_names
data_df = pd.DataFrame(data.data, columns=data.feature_names)

In [39]:
data_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### Train a Model
After loading and cleaning the data, split the datapoints into training and test sets. Assemble separate datasets for the full sample and the test data.

In [32]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    breast_cancer_data.data, breast_cancer_data.target, test_size=0.2, random_state=0)

Train a RF (Random-Forest) classifier on the training data

In [33]:
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [34]:
# Prepare the train_data and test_data
train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### Create Model and Data Insights

In [17]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

Defining an "identity_feature". An identity feature in Microsoft's Responsible AI Toolkit helps detect and mitigate bias in sensitive attributes like race, gender, and age for fairness. 

Marking a feature as an identity feature can trigger specific analyses within the toolkit, such as:

- Fairness assessment: Evaluating if certain identity groups are disproportionately affected by model decisions
- Bias detection: Highlighting patterns of bias in model performance across identity groups
- Mitigating unfairness: Providing tools to adjust or retrain models to improve fairness with respect to the identity features

In [15]:
from responsibleai.feature_metadata import FeatureMetadata
# Add 's1' as an identity feature
feature_metadata = FeatureMetadata(
    identity_feature_name=None, 
    categorical_features=[], 
    dropped_features=[])

RAIInsights accepts the model, the train dataset, the test dataset, the target feature string and the task type string as its arguments.

In [22]:
rai_insights = RAIInsights(
    model, 
    train_data, 
    test_data, 
    target_feature, 
    'classification',
    feature_metadata=feature_metadata)

Add the components of the toolbox that are focused on **Model assessment and Decision-making.**

- Explanations (.explainer): Clarifies how model features influence predictions using algorithms like SHAP or LIME
- Error Analysis (.error_analysis): Identifies subgroups of data where the model makes the most mistakes, helping with troubleshooting
- Counterfactuals (.counterfactual): Generates "what-if" scenarios, showing how feature changes affect outcomes
- Causal Inference (.causal): Analyzes whether certain features (e.g., bmi, bp) directly cause changes in the outcome

In [23]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the range that their label should fall under, 
# and a list of strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=20, desired_class="opposite")
# Causal Inference: determines causation between features
rai_insights.causal.add(
    treatment_features=['mean_radius', 'mean_texture'])  # Example features

Once all the desired components have been loaded, we compute insights on the test set

In [24]:
rai_insights.compute()

Causal Effects
Current Status: Generating Causal Effects.
Current Status: Finished generating causal effects.
Time taken: 0.0 min 7.871675400005188 sec
Counterfactual
Current Status: Generating 20 counterfactuals for 114 samples


 39%|███▉      | 45/114 [01:34<01:47,  1.56s/it]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 01 sec


 48%|████▊     | 55/114 [01:46<01:11,  1.22s/it]

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 01 sec


100%|██████████| 114/114 [04:02<00:00,  2.13s/it]


Current Status: Generated 20 counterfactuals for 114 samples.
Time taken: 4.0 min 6.155787600029726 sec
Error Analysis
Current Status: Generating error analysis reports.
Current Status: Finished generating error analysis reports.
Time taken: 0.0 min 0.7654632000485435 sec
Explanations
Current Status: Explaining 30 features
Current Status: Explained 30 features.
Time taken: 0.0 min 0.2095359999802895 sec


Visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab

In [25]:
ResponsibleAIDashboard(rai_insights)

ResponsibleAI started at http://localhost:8710


Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.


## Visualize
### [Optional] Load the interpretability visualization dashboard

### Explain predictions on your local machine

In [ ]:
breast_cancer_feature_names = list(X.columns)

In [ ]:
# 1. Using SHAP TabularExplainer
explainer = TabularExplainer(model,
                             X_train,
                             features=breast_cancer_feature_names,
                             classes=classes)

In [ ]:
from raiwidgets import ExplanationDashboard